In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()
!pip install torch
!pip install torchvision
!pip install d2l==1.0.0b0
!conda install -c conda-forge torchmetrics

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 66.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 62.1 MB/s eta 0:00:00
     ━━━━━

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - torchmetrics


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |       2_kmp_llvm           6 KB  conda-forge
    ca-certificates-2022.12.7  |       ha878542_0         143 KB  conda-forge
    certifi-2022.12.7          |     pyhd8ed1ab_0         147 KB  conda-forge
    colorama-0.4.6             |     pyhd8ed1ab_0          25 KB  conda-forge
    conda-23.1.0               |   py38h578d9bd

In [ ]:
!pip install wandb -qU
import wandb
wandb.login()

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
import os
import torch
import torchvision
from torchvision import transforms
from torch import nn, Tensor
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchsummary import summary
from d2l import torch as d2l
from pathlib import Path
import sys
import nibabel as nib
import numpy as np
import gc
import skimage
import h5py
import math
import PIL
from PIL import Image

/usr/local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
class FocalLoss(nn.Module):
    def __init__(self, weight0=1, weight1=1, gamma=0):
        super().__init__()

        self.weight0 = weight0
        self.weight1 = weight1
        self.gamma = gamma

    def forward(self, input, target):
        loss = 0

        #Takes negative average loss over each element of input
        #Loss = ln(prediction) * (absolute loss ^ gamma) * class weight
        #prediction is the predicted likelihood that the correct label is true
        for i, el in enumerate(input):
            if target[i] == 1:
                loss += torch.log(el) * (abs(1 - el) ** self.gamma) * self.weight1
            else:
                loss += torch.log(1 - el) * (abs(0 - el) ** self.gamma) * self.weight0

        return -1 * loss / len(input)

In [3]:
class BalancedCELoss(nn.Module):
    def __init__(self, weight0=1, weight1=1):
        super().__init__()

        self.weight0 = weight0
        self.weight1 = weight1

    def forward(self, input, target):
        loss = 0

        #Takes negative average loss over each element of input
        #Loss = ln(prediction) * class weight
        #prediction is the predicted likelihood that the correct label is true
        for i, el in enumerate(input):
            if target[i] == 1:
                loss += torch.log(el) * self.weight1
            else:
                loss += torch.log(1 - el) * self.weight0

        return -1 * loss / len(input)

In [4]:
class LITSBinaryDataset(Dataset):
    def __init__(self, fileName):
        super().__init__()

        #Keeps a file pointer open throughout use
        self.file = h5py.File(fileName, 'r')

        #Precalculates length to reduce training computations
        self.length = len(list(self.file.keys()))

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        data = self.file["Slice" + str(idx)]["Slice"]
        label = self.file["Slice" + str(idx)].attrs.get("ImageLabel")

        result = []

        #Returns list containing slice data and image label
        #Does not currently return segmentation data, will need to implement for decoder
        result.append(torch.Tensor(data[...]).unsqueeze(0))
        result.append(torch.Tensor(label).squeeze(0))

        return result

    def closeFile(self):
        #Closes file once dataset is no longer being used
        #Do not use class instance after this function is called
        self.file.close()

In [5]:
class convBlock(nn.Module):
    def __init__(self, inChannels, outChannels, batchNorm, strides, layerMean, layerDev) -> None:
        super().__init__()

        #Uses 2 convolutional layers for each block
        self.conv1 = nn.Conv2d(inChannels, outChannels, kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(outChannels, outChannels, kernel_size=3, padding=1)

        #Initializes convolutional layers using hyperparameters for mean and standard deviation
        nn.init.normal_(self.conv1.weight, mean=layerMean, std=layerDev)
        nn.init.normal_(self.conv2.weight, mean=layerMean, std=layerDev)

        if(batchNorm):
            self.bn1 = nn.BatchNorm2d(outChannels)
        else:
            self.bn1 = False

    def forward(self, X):
        Y = self.conv1(X)

        if(self.bn1):
            Y = self.bn1(Y)

        Y = F.relu(Y)

        return torch.Tensor(F.relu(self.conv2(Y)))

In [6]:
class EncoderBlock(nn.Module):
    def __init__(self, inChannels, outChannels, strides) -> None:
        super().__init__()

        #Creates a convBlock with batch norm and max pooling layer, directly from UNet paper
        self.conv = convBlock(inChannels, outChannels, True, strides, 0, 0.025)
        self.pool = nn.MaxPool2d(2, stride=strides)

    def forward(self, X):

        Y = self.conv.forward(X)

        #Will eventually need to return data directly from conv as well as max pooling for the skip connections
        #return torch.Tensor(self.pool(Y)), Y
        return self.pool(Y)

In [7]:
#Not fully implemented yet, waiting until encoder section performs well enough
class DecoderBlock(nn.Module):
    def __init__(self, inChannels, outChannels, strides) -> None:
        super().__init__()

        self.convTrans = nn.ConvTranspose2d(inChannels, outChannels, 2, stride=strides, padding=1)
        self.conv = convBlock(outChannels, outChannels, True, strides, 0, 0.25)

    def forward(self, X, skipFeatures):
        Y = self.convTrans(X)
        Y = torch.cat(X, skipFeatures)
        return self.conv(Y)

In [49]:
class EncoderNetwork(nn.Module):
    def __init__(self, channels, strides, dropout, device) -> None:
        super().__init__()

        self.device = device

        #Creates a list of encoder blocks w/ in and out channels specified by parameter
        self.blocks = []
        for i in range(len(channels)):
            if i == 0:
                self.blocks.append(EncoderBlock(1, channels[i], 1).to(device))
            else:
                self.blocks.append(EncoderBlock(channels[i - 1], channels[i], 1).to(device))

        #Creates dropout layer and classification branch
        self.dropout = nn.Dropout(dropout).to(device)
        self.classification = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(), nn.Linear(channels[-1], 1), nn.Sigmoid()).to(device)

    def forward(self, X):
        y = X.to(self.device)

        for block in self.blocks:
            y = block(y)

            #Need to check if we use dropout after every layer
            y = self.dropout(y)

        return self.classification(y)

In [9]:
def evaluate_accuracy(net, testIter, lossFunc, device=None):
    if isinstance(net, nn.Module):
        net.eval()
        if not device:
            device = next(iter(net.parameters())).device

    #Accuracy, number of samples, loss
    metric = d2l.Accumulator(3)

    with torch.no_grad():
        for X, y in testIter:
            X = X.to(device)
            y = y.to(device)

            yhat = net(X)

            prediction = torch.round(yhat).to(device)

            metric.add(d2l.accuracy(prediction, y) / torch.numel(y), torch.numel(y), lossFunc(yhat, y))

    return metric[0] / metric[1], metric[2] / len(testIter)

In [10]:
def train(net: nn.Sequential, trainIter, testIter, numEpochs, startEpoch, learnRate, batchSize, device: torch.device, startDim, epochsToDouble, modelFileName, epochsToSave, 
          useWandB=False, cosineAnnealing=True, restartEpochs=-1, progressive=False, lossFunc = nn.BCEWithLogitsLoss()):
    print(f"Training on {device}")
    
    net.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=learnRate)

    #Setting restartEpochs to a negative will use no warm restarts, otherwise will use warm restarts 
    if cosineAnnealing:
        if restartEpochs < 0:
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
        else:
            scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, restartEpochs, T_mult=1)

    numBatches = len(trainIter)
    bestValLoss = float('inf')

    currDim = startDim
    for epoch in range(startEpoch, numEpochs):
        #Progressive learning
        if (epoch + 1) % epochsToDouble == 0 and progressive == 1:
            currDim *= 2
        #Reverse progressive learning
        elif (epoch + 1) % epochsToDouble == 0 and progressive == 2:
            currDim /= 2
        
        net.train()
        
        #Loss, accuracy
        metric = d2l.Accumulator(2)

        for i, (X, y) in enumerate(trainIter):
            optimizer.zero_grad()
            y = y.to(device)

            if progressive > 0:
                #If using progressive learning, downsamples image to the current dimension
                X = F.interpolate(X, size=int(currDim))

            X = X.to(device)

            yhat = net(X).squeeze(1)

            prediction = torch.round(yhat).to(device)

            l = lossFunc(yhat, y)

            #print(f"Loss: {l.item()} Predictions: {yhat.tolist()} Labels: {y.tolist()}")

            l.backward()
            optimizer.step()

            if cosineAnnealing:
                scheduler.step(epoch + i / numBatches)

            metric.add(l, d2l.accuracy(prediction, y) / batchSize)

        #Checkpoints model
        if (epoch + 1) % epochsToSave == 0:
            torch.save(net.state_dict(), modelFileName + "Epoch" + str(epoch))

        validationAcc, validationLoss = evaluate_accuracy(net, testIter, lossFunc, device=device)

        #Overwrites previous best model based on validation accuracy
        if validationLoss < bestValLoss:
            bestValLoss = validationLoss
            torch.save(net.state_dict(), modelFileName + "BestLoss")

        print(f"Train Acc: {metric[1] / numBatches} Validation Acc: {validationAcc} Train Loss: {metric[0] / numBatches} Validation Loss: {validationLoss}")

        #Externally logs epoch info to WandB
        if useWandB:
            wandb.log({"Train Acc": metric[1] / numBatches,
                    "Validation Acc": validationAcc,
                    "Train Loss": metric[0] / numBatches,
                    "Validation Loss": validationLoss
                    })

In [11]:
#Hyperparameters and training modifications
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#device = torch.device("cpu")

modelName = "LiverClassifierBaseline"
fileSaveName = "/content/drive/MyDrive/" + modelName

#Use if starting from a checkpoint
startEpoch = 0

useWandB = False

batchSize = 8
learnRate = 0.001
epochs = 100
dropout = 0.15

#Progressive training parameters
startDim = 32
epochsToDouble = 25

#0: not progressive, 1: progressive, 2: reverse progressive
progressive = 1

#Checkpointing
epochsToSave = 10

#Learn rate scheduling parameters
cosineAnnealing = True
cosineRestartEpochs = 10

#lossFunc = BalancedCELoss(weight0=1, weight1=1.5)
lossFunc = FocalLoss(weight0=0.2, weight1=0.8, gamma=2)
#lossFunc = nn.BCEWithLogitsLoss()

In [46]:
#Load model
networkFileName = ""

net = EncoderNetwork([16, 32, 64, 128, 256], 1, dropout, device)
#print(summary(net, (1, 256, 256)))

#Loads model from file if using a pretrained version
if networkFileName != "":
    net.load_state_dict(torch.load(networkFileName))

net = net.to(device)

print("Intialized model")

Intialized model


In [22]:
#Load Datasets
trainDataset = LITSBinaryDataset("drive/MyDrive/MachineLearningResearch/Datasets/PartialTrainDataset.hdf5")
validationDataset = LITSBinaryDataset("drive/MyDrive/MachineLearningResearch/Datasets/ValidationDataset.hdf5")
testDataset = LITSBinaryDataset("drive/MyDrive/MachineLearningResearch/Datasets/TestDataset.hdf5")

trainIter = DataLoader(trainDataset, batch_size=batchSize, shuffle=True)
validationIter = DataLoader(validationDataset, batch_size=batchSize)
testIter = DataLoader(testDataset, batch_size=batchSize)

print("Dataset loaded")

Dataset loaded


In [47]:
#Train Model
gc.collect()

if useWandB:
    wandb.init(project="LiverClassifier1",
            name=modelName,
            config={
                "BatchSize":batchSize,
                "LearnRate":learnRate,
                "Epochs":epochs,
                "StartDimension":startDim,
                "EpochsToDouble":epochsToDouble
            })

train(net, trainIter, validationIter, epochs, startEpoch, learnRate, batchSize, device, startDim, epochsToDouble, fileSaveName, epochsToSave, useWandB=useWandB, 
      cosineAnnealing=False, restartEpochs=cosineRestartEpochs, progressive=progressive, lossFunc=lossFunc)

if useWandB:
    wandb.finish()

Training on cuda
Train Acc: 0.4006849315068493 Validation Acc: 0.24496644295302014 Train Loss: 0.06682059648510529 Validation Loss: 0.06052293304942156
Train Acc: 0.4006849315068493 Validation Acc: 0.24496644295302014 Train Loss: 0.06638380303366544 Validation Loss: 0.06112554963482054


KeyboardInterrupt: ignored

In [48]:
#Evaluate Model
print(f"Model: {modelName}")

trainAcc, trainLoss = evaluate_accuracy(net, trainIter, lossFunc, device=device)
print(f"Train Accuracy: {trainAcc} Train Loss: {trainLoss}")

validationAcc, validationLoss = evaluate_accuracy(net, validationIter, lossFunc, device=device)
print(f"Validation Accuracy: {validationAcc} Validation Loss: {validationLoss}")

testAcc, testLoss = evaluate_accuracy(net, testIter, lossFunc, device=device)
print(f"Test Accuracy: {testAcc} Test Loss: {testLoss}")

Model: LiverClassifierBaseline
Train Accuracy: 0.4013722126929674 Train Loss: 0.07085686452584723
Train Accuracy: 0.24496644295302014 Train Loss: 0.060490624390934646
Test Accuracy: 0.5031766200762389 Test Loss: 0.07727372571073397


In [ ]:
#Close datasets
trainDataset.closeFile()
validationDataset.closeFile()
testDataset.closeFile()